In [1]:
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import curve_fit
from scipy.stats import norm, stats

import sys
sys.path.append('/home/users/hswanson13/tbanalysis/') #stupid me
import run_analysis as ra
from tb_plotting import plotting_utilities as pu
from os import path
from datetime import datetime
import analysis_utilities as au

from os import path
import uproot
import yahist


In [5]:
import importlib
# importlib.reload(ra)

def linear(x, m, b):
    #m, b = p0
    return m*x + b

def get_x(m, b, y):
    return (y - b) / m

def add_old_clock(tree, plot_num=None):
    channel = tree['channel'].array()
    time = tree['time'].array()
    nSamples = len(time)
    clocks = channel[:,2] # Hardcoded clock channel
    triggers = channel[:,1] # Hardcoded trigger channel
    times = np.array(time[:,0])*10**9
    # breakpoint()
    clock = np.array(clocks)

    if plot_num is not None:
        fig, ax = plt.subplots(1,1, figsize=(16,8))
        ax.scatter(times[plot_num], clock[plot_num], color='lightgrey')
    
    minima = np.tile(np.min(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    maxima = np.tile(np.max(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    amp_fraction = 20 # %
    amp = minima + np.abs(minima - maxima)*amp_fraction/100
    min_scale = np.abs(maxima - minima)/10.0
    clock_diff = np.diff(clock, append=0)
    clock_diff_mask = clock_diff > min_scale
    # true after indices
    check_prior_fall = clock_diff < -min_scale
    prior_indices = np.argmax(check_prior_fall, axis=1)
    prior_fall_mask = np.arange(check_prior_fall.shape[1]) >= prior_indices[:, None]
    # breakpoint()
    global_mask = clock_diff_mask & prior_fall_mask

    if plot_num is not None:
        tcdiff = ak.Array([sublist[sublist != 0] for sublist in np.where(clock_diff_mask, times, 0)])
        ccdiff = ak.Array([sublist[sublist != 0] for sublist in np.where(clock_diff_mask, clock, 0)])

        tpfm = ak.Array([sublist[sublist != 0] for sublist in np.where(prior_fall_mask, times, 0)])
        cpfm = ak.Array([sublist[sublist != 0] for sublist in np.where(prior_fall_mask, clock, 0)])

        ax.scatter(tpfm[plot_num], cpfm[plot_num], color="deeppink", label="Prior Fall Mask")    
        ax.scatter(tcdiff[plot_num], ccdiff[plot_num], color="darkblue", label="clock_diff_mask")    

    # breakpoint()
    times = np.where(global_mask, times, 0)
    clock = np.where(global_mask, clock, 0)
    
    # delete 0 values for each row
    # breakpoint()
    times = ak.Array([sublist[sublist != 0] for sublist in times])
    clock = ak.Array([sublist[sublist != 0] for sublist in clock])

    # if plot_num is not None:
    #     ax.scatter(times[plot_num], clock[plot_num], color="#8024AB")

    #print(times)
    # breakpoint()
    time_slope = times[:,1] - times[:,0] #x run
    clock_slope = clock[:,1] - clock[:,0] #y rise
    slope = clock_slope / time_slope #y rise / x run
    ybias = clock[:,0] - slope*times[:,0]
    # calculate 20% of the amplitude
    amp = (minima + np.abs(minima - maxima)*amp_fraction/100)[:,0]
    clock_timestamp = np.array((amp - ybias) / slope)

    if plot_num is not None:
        ax.scatter([times[:,0][plot_num], times[:,1][plot_num]], [clock[:,0][plot_num], clock[:,1][plot_num]], marker='x', color="green", s=400)
        ax.axline((times[:,0][plot_num], clock[:,0][plot_num]), slope=slope[plot_num], color="green")
        ax.axline((times[:,1][plot_num], clock[:,1][plot_num]), slope=slope[plot_num], color="green", linewidth=3)
        ax.vlines(clock_timestamp[plot_num], ymin=minima[plot_num][0], ymax=maxima[plot_num][0],color='black', label=f"Clock Timestamp={clock_timestamp[plot_num]:.5}", linestyle="dotted", linewidth=1)
        ax.hlines(amp[plot_num],xmin=-25, xmax=25,color='r', linestyles="--", label=f"amp={amp[plot_num]:.5}")
        ax.hlines(minima[plot_num],xmin=-25, xmax=25,color='maroon', linestyles="dashed", label=f"minima={minima[plot_num][0]:.5}")
        ax.hlines(maxima[plot_num],xmin=-25, xmax=25,color='maroon', linestyles="dashed", label=f"maxima={maxima[plot_num][0]:.5}")
        ax.set_xlabel("time (ns)")
        ax.set_ylabel("Clock Voltage")
        ax.set_title("Clock Diff Mask")
        ax.grid()
        ax.legend(loc=1, prop={'size': 12}, frameon=True,fancybox=True)
    return clock_timestamp, slope

def add_old_period(events, amp_fraction=20, plot_num=None):
    def two_pnt_fit(times, clocks, edg_idx, amp):
        frst_times = times[:,edg_idx,0]
        scnd_times = times[:,edg_idx,1]

        frst_clocks = clocks[:,edg_idx,0]
        scnd_clocks = clocks[:,edg_idx,1]

        t_run = scnd_times - frst_times
        c_rise = scnd_clocks - frst_clocks
        slope = c_rise / t_run
        ybias = frst_clocks - slope*frst_times
        # calculate 20% of the amplitude
        #amp = (minima + np.abs(minima - maxima)*amp_fraction/100)[:,0]
        c_timestamp = np.array((amp[:,0] - ybias) / slope)
        return c_timestamp, slope

    channel = events['channel']
    time = events['time']
    clocks = channel[:,2] # Hardcoded clock channel
    times = np.array(time[:,0])*10**9
    clock = np.array(clocks)

    if plot_num is not None:
        fig, ax = plt.subplots(1,1, figsize=(16,8))
        ax.scatter(times[plot_num], clock[plot_num], color='lightgrey')

    minima = np.tile(np.min(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    maxima = np.tile(np.max(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    amp = minima + np.abs(minima - maxima)*amp_fraction/100

    min_scale = np.abs(maxima - minima)/10.0

    clock_diff = np.diff(clock, append=0)
    rising_edge_mask = clock_diff > min_scale #to get all rising edges min_scale is a positive number

    all_rising_times = np.where(rising_edge_mask, times, 0)
    all_rising_clocks = np.where(rising_edge_mask, clock, 0)
    all_rising_times = ak.Array([sublist[sublist != 0] for sublist in all_rising_times])
    all_rising_clocks = ak.Array([sublist[sublist != 0] for sublist in all_rising_clocks])

    t_diffs = [np.diff(t_e) for t_e in all_rising_times]
    cut_points = [np.where(diff > 5)[0]+1 for diff in t_diffs]
    
    split_times = [np.split(t_e, ct_p) for t_e, ct_p in zip(all_rising_times, cut_points)]
    split_clocks = [np.split(t_e, ct_p) for t_e, ct_p in zip(all_rising_clocks, cut_points)]
    
    nmask = ak.num(split_times, axis=2) > 2 #keep only edges with more than 2 points point, doing keeping one than mask on biggest points is best practice lets try this tho

    rising_times = ak.Array(split_times)[nmask]
    rising_clocks = ak.Array(split_clocks)[nmask]

    #one could maybe do another mask that only keeps two arrays with the most points

    clock_edge_0, frst_slope = two_pnt_fit(rising_times, rising_clocks, 0, amp)
    clock_edge_1, scnd_slope = two_pnt_fit(rising_times, rising_clocks, 1, amp)

    Ts = clock_edge_1 - clock_edge_0

    #closest edges to 0
    clk_nearest_zero = np.where((np.abs(clock_edge_0) <= np.abs(clock_edge_1)), clock_edge_0, clock_edge_1 )

    if plot_num is not None:
        #MAX MIN VOLTAGE AND AMP
        ax.hlines(amp[plot_num],xmin=-25, xmax=25,color='r', linestyles="--", label=f"amp={amp[plot_num][0]:.5}")
        ax.hlines(minima[plot_num],xmin=-25, xmax=25,color='maroon', linestyles="dashed", label=f"minima={minima[plot_num][0]:.5}")
        ax.hlines(maxima[plot_num],xmin=-25, xmax=25,color='maroon', linestyles="dashed", label=f"maxima={maxima[plot_num][0]:.5}")

        #THE FIRST AND SECOND RISING EDGE POINTS
        ax.scatter(all_rising_times[plot_num], all_rising_clocks[plot_num], color='black')
        ax.scatter(rising_times[plot_num][0], rising_clocks[plot_num][0], color='deeppink', label="first rising edge")
        ax.scatter(rising_times[plot_num][1], rising_clocks[plot_num][1], color='blue', label="second rising edge")

        #SELECTED POINTS FOR FIT
        #ax.scatter([times[:,0][plot_num], times[:,1][plot_num]], [clock[:,0][plot_num], clock[:,1][plot_num]], marker='x', color="green", s=400)
        ax.scatter([rising_times[plot_num][0][0], rising_times[plot_num][0][1]], [rising_clocks[plot_num][0][0], rising_clocks[plot_num][0][1]], marker='x', color="green", s=400)
        ax.scatter([rising_times[plot_num][1][0], rising_times[plot_num][1][0]], [rising_clocks[plot_num][1][0], rising_clocks[plot_num][1][1]], marker='x', color="green", s=400)
        #FITTED LINE ON FIRST AND SECOND
        ax.axline((rising_times[plot_num,0,0], rising_clocks[plot_num,0,0]), slope=frst_slope[plot_num], color="green")
        ax.axline((rising_times[plot_num,1,1], rising_clocks[plot_num,1,1]), slope=scnd_slope[plot_num], color="green", linewidth=3)

        #FIRST AND SECOND TIME STAMP
        ax.vlines(clock_edge_0[plot_num], ymin=minima[plot_num][0], ymax=maxima[plot_num][0],color='black', label=f"First Time Stamp={clock_edge_0[plot_num]:.5}", linestyle="dotted", linewidth=1)
        ax.vlines(clock_edge_1[plot_num], ymin=minima[plot_num][0], ymax=maxima[plot_num][0],color='black', label=f"Second Time Stamp={clock_edge_1[plot_num]:.5}", linestyle="dotted", linewidth=1)
        
        ax.set_xlabel("time (ns)")
        ax.set_ylabel("Clock Voltage")

        ax.set_xlabel("time (ns)")
        ax.set_ylabel("Clock Voltage")
        ax.legend(loc=1, prop={'size': 12}, frameon=True,fancybox=True)

    return Ts, clock_edge_0, clock_edge_1, clk_nearest_zero, frst_slope, scnd_slope

In [ ]:
#Aram clock plot
data_path = '/ceph/cms/store/user/iareed/ETL/DESY_Mar24'
run_file = 'run_5051.root'
event_num = 67


from scipy.optimize import curve_fit

def linear(x, *p0):
    a, b = p0
    return a + b*x

def plot_linear_fit(ax, x_data, y_data, p0, color='red', lw=2):
    # Do a simple gaussian fit
    try:
        coeff, var_martix = curve_fit(linear, x_data, y_data, p0=p0)
    except RuntimeError:
        coeff = p0
    ax.plot(
        x_data,
        linear(y_data, *coeff),
        color=color,
        label='Linear Fit\n slope: {:.2f} \n intercept: {:.3f}'.format(coeff[1],abs(coeff[0])),
        linewidth=lw
    )

def slope_on_slope_fit(ax, slope_hist):
    bin_centers = np.array(slope_hist.bin_centers)
    slopes_counts = np.array(slope_hist.counts)
    bin_cent_mask = (bin_centers > 1.3) & (bin_centers < 1.57)
    plot_linear_fit(ax, bin_centers[bin_cent_mask], slopes_counts[bin_cent_mask], [150, 100/0.3])


run_data = uproot.open( path.join(data_path, run_file) )['pulse']
_, desy_slopes = add_old_clock(run_data)#, plot_num=event_num)

for chunk in au.run_chunker([path.join(data_path, run_file)]):
    events = uproot.concatenate(chunk)
    pout = add_old_period(events)#, plot_num=event_num)


fig = plt.figure(constrained_layout=True,  figsize=(14, 6))
ax = plt.subplot(1,1,1)

slopes_frst = yahist.Hist1D(pout[-2]).normalize()
slopes_scnd = yahist.Hist1D(pout[-1]).normalize()
slopes_desy = yahist.Hist1D(desy_slopes).normalize()
#slope_on_slope_fit(ax, slopes_desy)

slopes_frst.plot(ax, label="First Rising Edge Slopes")
slopes_scnd.plot(ax, label="Second Rising Edge Slopes")
slopes_desy.plot(ax, label="DESY Rising Edge Slopes")


ax.set_xlabel("Rising Edge Slopes")
ax.set_ylabel("counts")

ax.legend()


# add_old_clock(run_data, plot_num=41)
# add_old_clock(run_data, plot_num=67)

# for chunk in au.run_chunker([path.join(data_path, run_file)]):
#     events = uproot.concatenate(chunk)
#     add_old_period(events, amp_fraction=20, plot_num=20)


In [9]:
import uproot
from os import path
import numpy as np
import awkward as ak
import sys
sys.path.append('/home/users/hswanson13/tbanalysis/') #stupid me
from utils import analysis as au

data_path = '/ceph/cms/store/user/iareed/ETL/DESY_Mar24'
run_file = 'run_5051.root'
#my_periods = []
frst_edg_clks = []
scnd_edg_clks = []
nearest_zero_clks = []

run_data = uproot.open( path.join(data_path, run_file) )['pulse']
aram_clock = add_old_clock(run_data)

desy_periods = []
for chunk in au.run_chunker([path.join(data_path, run_file)]):
    events = uproot.concatenate(chunk)

    desy_periods_chunk, frst_edg_clk, scnd_edg_clk, nearest_zero_clk = add_old_period(events)
    #my_periods += list(myp)
    frst_edg_clks += list(frst_edg_clk)
    scnd_edg_clks += list(scnd_edg_clk)
    nearest_zero_clks += list(nearest_zero_clk)

    desy_periods += list(desy_periods_chunk)



NameError: name 'au' is not defined

In [ ]:
#diff = np.array(my_clock) - aram_clock
#hist_diff = yahist.Hist1D(np.array(my_clock) - aram_clock)
hist_aram = yahist.Hist1D(aram_clock)
hist_frst = yahist.Hist1D(np.array(frst_edg_clks))
hist_scnd = yahist.Hist1D(np.array(scnd_edg_clks))
hist_near_0 = yahist.Hist1D(np.array(nearest_zero_clks))

#fig, axs = plt.subplots(2, 2)
fig = plt.figure(constrained_layout=True,  figsize=(14, 6))
gs = fig.add_gridspec(1, 4)
ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1])
ax3 = fig.add_subplot(gs[0,2])
ax4 = fig.add_subplot(gs[0,3])

hist_aram.plot(ax1, color="purple")
hist_frst.plot(ax2, color="blue")
hist_scnd.plot(ax3, color="green")
hist_near_0.plot(ax4, color="red")

fntsize=20
ax1.set_title("DESY Clock", fontsize=fntsize)
ax2.set_title("First Edge Clock", fontsize=fntsize)
ax3.set_title("Second Edge Clock", fontsize=fntsize)
ax4.set_title("Nearest Zero Clock", fontsize=fntsize)

ax1.set_xticks(np.arange(-15,15+5,5))
ax2.set_xticks(np.arange(-25,0+5,5))
ax3.set_xticks(np.arange(0,25+5,5))
ax4.set_xticks(np.arange(-15,15+5,5))

fntsize = 12
ax1.set_xlabel("t (ns)", fontsize=fntsize)
ax2.set_xlabel("t (ns)", fontsize=fntsize)
ax3.set_xlabel("t (ns)", fontsize=fntsize)
ax4.set_xlabel("t (ns)", fontsize=fntsize)

plt.show()

In [ ]:
hist = yahist.Hist1D(desy_periods, bins='auto', label=f'Mean Period: {np.mean(desy_periods):.5} ns, SDM: {np.std(desy_periods)*1000:.5} ps')

fig, ax = plt.subplots(1, 1, tight_layout=True)

hist.plot(ax, legend=True)
